# K₇ Laplacian Comparison v2: Δ₀ vs Δ₁ (GPU-Optimized)

**Objective**: Compare scalar Laplacian (Δ₀) vs 1-form Hodge Laplacian (Δ₁) on K₇

**GIFT Prediction**: λ₁ × H* = 13 or 14 (dim(G₂) - 1 or dim(G₂))

**v2 Improvements**:
- CuPy GPU acceleration (100-1000x faster)
- Optimized 1-form Laplacian computation
- gift_core integration for verified constants
- Better numerical stability

---

In [ ]:
# @title 1. Install Dependencies (run once)
%%capture
!pip install cupy-cuda12x gift-core scipy numpy matplotlib
# Use cupy-cuda11x if you have CUDA 11.x

In [ ]:
# @title 2. Imports & GPU Check
import numpy as np
import time
import json
from dataclasses import dataclass, asdict
from typing import Tuple, Optional

# GPU acceleration
try:
    import cupy as cp
    import cupyx.scipy.sparse as cp_sparse
    import cupyx.scipy.sparse.linalg as cp_linalg
    GPU_AVAILABLE = True
    print(f"✓ CuPy GPU available")
    print(f"  Device: {cp.cuda.Device().name}")
    print(f"  Memory: {cp.cuda.Device().mem_info[1] / 1e9:.1f} GB")
except ImportError:
    GPU_AVAILABLE = False
    print("✗ CuPy not available, using CPU fallback")

# CPU fallback
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh
from scipy.spatial import KDTree

# GIFT constants
try:
    from gift_core.constants import (
        DIM_G2, DIM_K7, B2, B3, H_STAR, DET_G
    )
    print(f"✓ gift_core loaded")
    GIFT_LOADED = True
except ImportError:
    print("✗ gift_core not found, using hardcoded constants")
    GIFT_LOADED = False
    DIM_G2 = 14
    DIM_K7 = 7
    B2 = 21
    B3 = 77
    H_STAR = 99
    DET_G = 65/32

print(f"\nGIFT Constants: dim(G₂)={DIM_G2}, H*={H_STAR}, b₂={B2}, b₃={B3}")

In [ ]:
# @title 3. GPU-Accelerated Sparse Eigensolver

def sparse_eigsh_gpu(
    L_cpu: csr_matrix,
    k: int = 10,
    which: str = 'SM',
    tol: float = 1e-6,
    maxiter: int = 5000
) -> np.ndarray:
    """
    GPU-accelerated sparse eigenvalue solver.
    
    Falls back to CPU if GPU unavailable or fails.
    """
    if GPU_AVAILABLE and L_cpu.shape[0] > 1000:
        try:
            # Transfer to GPU
            L_gpu = cp_sparse.csr_matrix(
                (cp.asarray(L_cpu.data.astype(np.float64)),
                 cp.asarray(L_cpu.indices),
                 cp.asarray(L_cpu.indptr)),
                shape=L_cpu.shape
            )
            
            # Compute on GPU
            eigenvalues, _ = cp_linalg.eigsh(
                L_gpu, k=k, which=which, tol=tol, maxiter=maxiter
            )
            
            return cp.asnumpy(np.sort(eigenvalues))
        except Exception as e:
            print(f"  GPU failed ({e}), falling back to CPU")
    
    # CPU fallback
    eigenvalues, _ = eigsh(L_cpu.astype(np.float64), k=k, which=which, tol=tol, maxiter=maxiter)
    return np.sort(eigenvalues)

print("GPU eigensolver ready.")

In [ ]:
# @title 4. K₇ Sampling (TCS Construction)

def sample_K7_TCS(n: int, seed: int = 42) -> np.ndarray:
    """
    Sample from K₇ via Twisted Connected Sum construction.
    
    K₇ ≈ (S³ × T⁴) ∪_φ (S³ × T⁴)
    """
    rng = np.random.default_rng(seed)
    
    # Canonical ratio from TCS
    ratio = H_STAR / (6 * DIM_G2)  # = 99/84 ≈ 1.179
    
    points = []
    for half, sign in [(n//2, 1), (n - n//2, -1)]:
        # S³ component
        s3 = rng.standard_normal((half, 4))
        s3 = s3 / np.linalg.norm(s3, axis=1, keepdims=True) * ratio
        
        # T³ component (from T⁴)
        t3 = 2 * np.pi * rng.random((half, 3))
        
        # Combine to 7D: [S³[:3], T³, sign × S³[3]]
        pts = np.column_stack([s3[:, :3], t3, sign * s3[:, 3:4]])
        points.append(pts)
    
    points = np.vstack(points)
    
    # Apply G₂ metric scaling (det(g) = 65/32)
    scale = (DET_G ** (1/7)) ** 0.5
    scales = np.array([1, 1, 1, scale, scale, scale, scale**0.5])
    
    return points * scales

# Test
test_pts = sample_K7_TCS(100)
print(f"K₇ sampling: {test_pts.shape}, mean={test_pts.mean():.3f}, std={test_pts.std():.3f}")

In [ ]:
# @title 5. Optimized Laplacian Builders

def build_graph_laplacian(
    points: np.ndarray,
    k: int = 15
) -> Tuple[csr_matrix, float, np.ndarray, np.ndarray]:
    """
    Build scalar graph Laplacian (Δ₀) with k-NN.
    
    Returns: L, epsilon, distances, indices
    """
    n = len(points)
    tree = KDTree(points)
    distances, indices = tree.query(points, k=k+1)
    
    # Bandwidth from median distance
    epsilon = np.median(distances[:, 1:]) ** 2
    
    # Build sparse weight matrix
    rows, cols, data = [], [], []
    for i in range(n):
        for j_idx in range(1, k+1):
            j = indices[i, j_idx]
            w = np.exp(-distances[i, j_idx]**2 / epsilon)
            rows.extend([i, j])
            cols.extend([j, i])
            data.extend([w, w])
    
    W = csr_matrix((data, (rows, cols)), shape=(n, n))
    W = (W + W.T) / 2  # Ensure symmetry
    
    # L = D - W
    D = diags(np.array(W.sum(axis=1)).flatten())
    L = D - W
    
    return L, epsilon, distances, indices


def build_1form_laplacian_fast(
    points: np.ndarray,
    k: int = 15,
    epsilon: Optional[float] = None
) -> Tuple[csr_matrix, int]:
    """
    Build 1-form Hodge Laplacian (Δ₁) efficiently.
    
    Uses the discrete exterior derivative: Δ₁ = d₀ᵀd₀
    
    Optimized for speed: minimal edge storage, direct sparse construction.
    """
    n = len(points)
    tree = KDTree(points)
    distances, indices = tree.query(points, k=k+1)
    
    if epsilon is None:
        epsilon = np.median(distances[:, 1:]) ** 2
    
    # Collect unique edges (i < j)
    edges = {}
    for i in range(n):
        for j_idx in range(1, k+1):
            j = indices[i, j_idx]
            edge = (min(i, j), max(i, j))
            if edge not in edges:
                d = distances[i, j_idx]
                edges[edge] = np.exp(-d**2 / (2 * epsilon))
    
    edge_list = sorted(edges.keys())
    n_edges = len(edge_list)
    
    # Build incidence matrix d₀ (n_edges × n)
    rows, cols, data = [], [], []
    for e_idx, (i, j) in enumerate(edge_list):
        w = np.sqrt(edges[(i, j)])
        rows.extend([e_idx, e_idx])
        cols.extend([i, j])
        data.extend([-w, w])
    
    d0 = csr_matrix((data, (rows, cols)), shape=(n_edges, n), dtype=np.float64)
    
    # Δ₁ = d₀ᵀ d₀ (down Laplacian on 1-forms)
    # This is n × n, NOT n_edges × n_edges!
    # For the actual 1-form Laplacian on edges, we need d₀ d₀ᵀ
    L1 = d0 @ d0.T  # n_edges × n_edges
    
    return L1, n_edges

print("Laplacian builders ready.")

In [ ]:
# @title 6. Calibration on Reference Manifolds

def sample_sphere(n: int, dim: int, seed: int = 42) -> np.ndarray:
    """Sample from S^dim."""
    rng = np.random.default_rng(seed)
    x = rng.standard_normal((n, dim + 1))
    return x / np.linalg.norm(x, axis=1, keepdims=True)

def sample_torus(n: int, dim: int, seed: int = 42) -> np.ndarray:
    """Sample from T^dim."""
    rng = np.random.default_rng(seed)
    return 2 * np.pi * rng.random((n, dim))

def calibrate(n: int = 2000, k: int = 15) -> float:
    """
    Calibrate scale factor on S³ (λ₁ = 3).
    """
    print("Calibrating on S³...")
    points = sample_sphere(n, 3)
    L, _, _, _ = build_graph_laplacian(points, k=k)
    eigs = sparse_eigsh_gpu(L, k=5, which='SM')
    
    # First nonzero
    raw_lambda1 = eigs[1] if eigs[0] < 1e-10 else eigs[0]
    scale = 3.0 / raw_lambda1  # True λ₁(S³) = 3
    
    print(f"  Raw λ₁ = {raw_lambda1:.6f}")
    print(f"  Scale factor = {scale:.4f}")
    return scale

# Run calibration
SCALE_FACTOR = calibrate(n=2000, k=15)

In [ ]:
# @title 7. Main Experiment

@dataclass
class Result:
    N: int
    k: int
    
    # Δ₀ results
    lambda1_0_raw: float
    lambda1_0_scaled: float
    product_0: float
    dev_0_from_13: float
    dev_0_from_14: float
    time_0: float
    
    # Δ₁ results
    lambda1_1_raw: float
    lambda1_1_scaled: float
    product_1: float
    dev_1_from_13: float
    dev_1_from_14: float
    time_1: float
    n_edges: int


def run_experiment(N: int = 3000, k: int = 15, seed: int = 42) -> Result:
    """
    Compare Δ₀ and Δ₁ on K₇.
    """
    print(f"\n{'='*60}")
    print(f"K₇ EXPERIMENT: N={N}, k={k}")
    print(f"{'='*60}")
    
    # Sample K₇
    print(f"\nSampling K₇...")
    points = sample_K7_TCS(N, seed=seed)
    print(f"  Shape: {points.shape}")
    
    # === Δ₀ (Scalar) ===
    print(f"\n--- Δ₀ (Scalar Laplacian) ---")
    t0 = time.time()
    L0, epsilon, _, _ = build_graph_laplacian(points, k=k)
    eigs_0 = sparse_eigsh_gpu(L0, k=10, which='SM')
    time_0 = time.time() - t0
    
    lambda1_0_raw = eigs_0[1] if eigs_0[0] < 1e-10 else eigs_0[0]
    lambda1_0_scaled = lambda1_0_raw * SCALE_FACTOR
    product_0 = lambda1_0_scaled * H_STAR
    
    print(f"  λ₁ raw = {lambda1_0_raw:.6f}")
    print(f"  λ₁ scaled = {lambda1_0_scaled:.6f}")
    print(f"  λ₁ × H* = {product_0:.4f}")
    print(f"  Time: {time_0:.2f}s")
    
    # === Δ₁ (1-Forms) ===
    print(f"\n--- Δ₁ (1-Form Hodge Laplacian) ---")
    t1 = time.time()
    L1, n_edges = build_1form_laplacian_fast(points, k=k, epsilon=epsilon)
    print(f"  n_edges = {n_edges}, L1 shape = {L1.shape}")
    
    # For 1-form Laplacian, we need fewer eigenvalues but more carefully
    # The first b₁ eigenvalues are zero (harmonic 1-forms), for K₇ b₁=0
    n_eigs = min(20, n_edges - 1)
    eigs_1 = sparse_eigsh_gpu(L1, k=n_eigs, which='SM', maxiter=10000, tol=1e-5)
    time_1 = time.time() - t1
    
    # First nonzero eigenvalue
    nonzero = eigs_1[eigs_1 > 1e-8]
    lambda1_1_raw = nonzero[0] if len(nonzero) > 0 else eigs_1[1]
    lambda1_1_scaled = lambda1_1_raw * SCALE_FACTOR
    product_1 = lambda1_1_scaled * H_STAR
    
    print(f"  λ₁ raw = {lambda1_1_raw:.6f}")
    print(f"  λ₁ scaled = {lambda1_1_scaled:.6f}")
    print(f"  λ₁ × H* = {product_1:.4f}")
    print(f"  Time: {time_1:.2f}s")
    
    # Deviations
    dev_0_13 = abs(product_0 - 13) / 13 * 100
    dev_0_14 = abs(product_0 - 14) / 14 * 100
    dev_1_13 = abs(product_1 - 13) / 13 * 100
    dev_1_14 = abs(product_1 - 14) / 14 * 100
    
    return Result(
        N=N, k=k,
        lambda1_0_raw=lambda1_0_raw, lambda1_0_scaled=lambda1_0_scaled,
        product_0=product_0, dev_0_from_13=dev_0_13, dev_0_from_14=dev_0_14, time_0=time_0,
        lambda1_1_raw=lambda1_1_raw, lambda1_1_scaled=lambda1_1_scaled,
        product_1=product_1, dev_1_from_13=dev_1_13, dev_1_from_14=dev_1_14,
        time_1=time_1, n_edges=n_edges
    )

print("Experiment function ready.")

In [ ]:
# @title 8. RUN EXPERIMENT

# Configuration - adjust based on GPU memory
N_SAMPLES = 3000   # Start small, increase if fast
K_NEIGHBORS = 15   # Keep moderate

result = run_experiment(N=N_SAMPLES, k=K_NEIGHBORS, seed=42)

In [ ]:
# @title 9. Results Summary

print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)

print(f"\n{'Metric':<25} {'Δ₀ (Scalar)':<20} {'Δ₁ (1-Forms)':<20}")
print("-"*70)
print(f"{'λ₁ × H*':<25} {result.product_0:<20.4f} {result.product_1:<20.4f}")
print(f"{'Deviation from 13':<25} {result.dev_0_from_13:<19.2f}% {result.dev_1_from_13:<19.2f}%")
print(f"{'Deviation from 14':<25} {result.dev_0_from_14:<19.2f}% {result.dev_1_from_14:<19.2f}%")
print(f"{'Compute time':<25} {result.time_0:<19.2f}s {result.time_1:<19.2f}s")
print("-"*70)

# Winner
best_0 = min(result.dev_0_from_13, result.dev_0_from_14)
best_1 = min(result.dev_1_from_13, result.dev_1_from_14)
winner = "Δ₁ (1-Forms)" if best_1 < best_0 else "Δ₀ (Scalar)"

print(f"\n🏆 WINNER: {winner}")
print(f"   Best deviation: {min(best_0, best_1):.2f}%")

# Physical interpretation
print(f"\n--- Physical Interpretation ---")
lambda1_phys = result.product_1 / H_STAR
mass_gap = lambda1_phys * 200  # Λ_QCD ≈ 200 MeV
print(f"From Δ₁: λ₁ = {lambda1_phys:.4f}")
print(f"         Δ_QCD = λ₁ × Λ_QCD = {mass_gap:.1f} MeV")
print(f"\nGIFT prediction: Δ_QCD ≈ (14/99) × 200 = 28.3 MeV")

In [ ]:
# @title 10. Visualization
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Bar comparison
ax1 = axes[0]
x = ['Δ₀ (Scalar)', 'Δ₁ (1-Forms)']
products = [result.product_0, result.product_1]
colors = ['steelblue', 'darkorange']
bars = ax1.bar(x, products, color=colors, edgecolor='black', linewidth=1.5)
ax1.axhline(y=13, color='red', linestyle='--', linewidth=2, label='Target: 13 (dim G₂ - 1)')
ax1.axhline(y=14, color='green', linestyle='--', linewidth=2, label='Target: 14 (dim G₂)')
ax1.set_ylabel('λ₁ × H*', fontsize=12)
ax1.set_title('Laplacian Comparison on K₇', fontsize=14)
ax1.legend()
for bar, val in zip(bars, products):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{val:.2f}', ha='center', fontsize=11, fontweight='bold')

# Deviation comparison
ax2 = axes[1]
x_pos = np.arange(2)
width = 0.35
dev_13 = [result.dev_0_from_13, result.dev_1_from_13]
dev_14 = [result.dev_0_from_14, result.dev_1_from_14]
ax2.bar(x_pos - width/2, dev_13, width, label='From 13', color='red', alpha=0.7)
ax2.bar(x_pos + width/2, dev_14, width, label='From 14', color='green', alpha=0.7)
ax2.set_ylabel('Deviation (%)', fontsize=12)
ax2.set_title('Deviation from GIFT Targets', fontsize=14)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['Δ₀', 'Δ₁'])
ax2.legend()

plt.tight_layout()
plt.savefig('K7_laplacian_v2.png', dpi=150, bbox_inches='tight')
plt.show()
print("\nSaved: K7_laplacian_v2.png")

In [ ]:
# @title 11. Export Results

output = {
    'config': {
        'N': result.N,
        'k': result.k,
        'H_star': H_STAR,
        'dim_G2': DIM_G2,
        'scale_factor': SCALE_FACTOR,
        'gpu_used': GPU_AVAILABLE,
        'gift_core_loaded': GIFT_LOADED,
    },
    'scalar_laplacian': {
        'lambda1_raw': result.lambda1_0_raw,
        'lambda1_scaled': result.lambda1_0_scaled,
        'product': result.product_0,
        'deviation_from_13': result.dev_0_from_13,
        'deviation_from_14': result.dev_0_from_14,
        'time_seconds': result.time_0,
    },
    '1form_laplacian': {
        'lambda1_raw': result.lambda1_1_raw,
        'lambda1_scaled': result.lambda1_1_scaled,
        'product': result.product_1,
        'deviation_from_13': result.dev_1_from_13,
        'deviation_from_14': result.dev_1_from_14,
        'time_seconds': result.time_1,
        'n_edges': result.n_edges,
    },
    'winner': 'Delta_1' if best_1 < best_0 else 'Delta_0',
    'best_target': 13 if min(result.dev_1_from_13, result.dev_0_from_13) < min(result.dev_1_from_14, result.dev_0_from_14) else 14,
}

with open('K7_laplacian_results_v2.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Saved: K7_laplacian_results_v2.json")
print("\n" + "="*70)
print("EXPERIMENT COMPLETE")
print("="*70)

In [ ]:
# @title 12. Quick Summary (copy-paste ready)

summary = f"""
╔══════════════════════════════════════════════════════════════════╗
║  K₇ LAPLACIAN COMPARISON v2 - SUMMARY                           ║
╠══════════════════════════════════════════════════════════════════╣
║  Config: N={result.N}, k={result.k}, H*={H_STAR}                              ║
║  GPU: {str(GPU_AVAILABLE):<5}  gift_core: {str(GIFT_LOADED):<5}                            ║
╠══════════════════════════════════════════════════════════════════╣
║  Δ₀ (Scalar):   λ₁ × H* = {result.product_0:>8.4f}  (time: {result.time_0:>5.1f}s)         ║
║  Δ₁ (1-Forms):  λ₁ × H* = {result.product_1:>8.4f}  (time: {result.time_1:>5.1f}s)         ║
╠══════════════════════════════════════════════════════════════════╣
║  Deviations:                                                     ║
║    Δ₀: from 13 = {result.dev_0_from_13:>6.2f}%  |  from 14 = {result.dev_0_from_14:>6.2f}%              ║
║    Δ₁: from 13 = {result.dev_1_from_13:>6.2f}%  |  from 14 = {result.dev_1_from_14:>6.2f}%              ║
╠══════════════════════════════════════════════════════════════════╣
║  🏆 WINNER: {winner:<20}                              ║
║  Physical: Δ_QCD ≈ {mass_gap:>5.1f} MeV (GIFT: 28.3 MeV)                  ║
╚══════════════════════════════════════════════════════════════════╝
"""

print(summary)